# Hadoop mapReduce on US state school districts financial data

### The data 

In [4]:
import pandas as pd
df = pd.read_csv('schools.txt', sep="\t")
df.head()

,STATE,NAME,YRDATA,TOTALREV,TFEDREV,TSTREV,TLOCREV,TOTALEXP,TCURELSC,TCURINST,TCURSSVC,TCUROTH,NONELSEC,TCAPOUT
0,Alabama,AUTAUGA CO SCH DIST,2000,48294,3344,34783,10167,49796,42037,26648,12921,2468,752,4761
1,Alabama,BALDWIN CO SCH DIST,2000,150579,10578,78199,61802,138009,128009,81035,40483,6491,2792,5378
2,Alabama,BARBOUR CO SCH DIST,2000,11773,1628,8225,1920,11801,10244,5837,3480,927,265,1083
3,Alabama,EUFAULA CTY SCH DIST,2000,19780,1727,11936,6117,18575,16709,10389,5085,1235,451,1412
4,Alabama,BIBB CO SCH DIST,2000,21020,2531,15267,3222,21306,19004,11934,5604,1466,498,1396


## 1st MapReducer for calculating the school district with the max revenue for each state followed by the year that it achieved it

## Mapper

In [ ]:
import sys
for line in sys.stdin:
    data = line.strip().split("\t")
    if len(data) == 14:
        state, school_dist, year, revenue, federal_rev, state_rev, local_rev, expend,
        tcureklsc, tcurinst,tcurssv,tcurth,nonelsec,tcapout = data
        print "{0}\t{1}\t{2}\t{3}".format(state,  school_dist, revenue,year)



## Reducer

In [ ]:
#!/usr/bin/python

import sys

oldKey = None
maxrev = 0
school = None

for line in sys.stdin:
    data_mapped = line.strip().split("\t")
    if len(data_mapped) != 4:
        continue           

    newKey, school_dist, revenue, year  = data_mapped

    if oldKey and oldKey != newKey:
        print oldKey , "\t", maxrev, "\t", school, "\t", yr
        oldKey = newKey;
        maxrev = 0
        school = None
        yr = 0
   
    oldKey = newKey
    if maxrev < float(revenue):
	    maxrev = float(revenue)
        school = school_dist
        yr = int(year)  
        
if oldKey != None:
    print oldKey , "\t" , maxrev, "\t", school, "\t", yr

## Result

In [16]:
data = pd.read_csv('result_schools_year_state', sep="\t", names = ["State", "Max Revenue", "School District", "Year"])
pd.options.display.float_format = '{:.0f}'.format 
data

,State,Max Revenue,School District,Year
0,Alabama,618181,MOBILE COUNTY SCHOOL DISTRICT,2008
1,Alaska,914050,ANCHORAGE SCHOOL DISTRICT,2015
2,Arizona,635243,MESA UNIFIED SCHOOL DIST,2008
3,Arkansas,371708,LITTLE ROCK SCH DISTRICT,2012
4,California,9695990,LOS ANGELES UNIF SCH DIST,2009
5,Colorado,1187223,DENVER SCH DIST 1,2015
6,Connecticut,521675,HARTFORD PUBLIC SCHOOLS,2015
7,Delaware,318466,CHRISTINA SCHOOL DISTRICT,2015
8,District of Columbia,1382282,DISTRICT OF COLUMBIA PUBLIC SCHOOLS,2015
9,Florida,3959408,MIAMI-DADE COUNTY PUBLIC SCHOOL DISTRICT,2008


## 2nd MapReducer for calculating the overall revenue for each State

## Mapper

In [ ]:
import sys
for line in sys.stdin:
    data = line.strip().split("\t")
    if len(data) == 14:
        state, school_dist, year, revenue, federal_rev, state_rev, local_rev, expend, tcureklsc, tcurinst,tcurssv,tcurth,nonelsec,tcapout = data
        print "{0}\t{1}".format(state,  revenue)

## Reducer

In [ ]:
import sys

revenueTotal = 0
oldKey = None



for line in sys.stdin:
    data_mapped = line.strip().split("\t")
    if len(data_mapped) != 2:
       
        continue

    newKey, revenue = data_mapped

    if oldKey and oldKey != newKey:
        print oldKey, "\t", revenueTotal
        oldKey = newKey;
        revenueTotal = 0

    oldKey = newKey
    revenueTotal += float(revenue)

if oldKey != None:
    print oldKey, "\t", revenueTotal


## Result

In [15]:
data = pd.read_csv('result_sum_revenue_perstate.txt', sep="\t",names = ["State", "Revenue"])

data


,State,Revenue
0,Alabama,128468921
1,Alaska,39456709
2,Arizona,152674940
3,Arkansas,83172124
4,California,1279502695
5,Colorado,148143046
6,Connecticut,171805514
7,Delaware,31235363
8,District of Columbia,23480558
9,Florida,481996387
